In [0]:
access_key = 'xxxxxxxxx'
secret_key = 'xxxxxxxx'
aws_region = "us-east-2"

In [0]:
import time
import pandas as pd

# Configurar acesso ao S3
import boto3
from io import StringIO

In [0]:
# Inicializar o cliente S3
s3_client = boto3.client(
    's3',
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    region_name=aws_region
)


In [0]:
# Ler o arquivo CSV diretamente do S3
bucket_name = "financial-dataset-ada"
file_key = "transactions_data.csv"
response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
file_content = response["Body"].read().decode("utf-8")

In [0]:
def read_file(file_content):
    df = pd.read_csv(StringIO(file_content))
    return df

In [0]:
# Função para medir o tempo de execução
def measure_time(func, *args):
    start_time = time.time()
    result = func(*args)
    end_time = time.time()
    print(f"Tempo de execução para {func.__name__}: {end_time - start_time:.4f} segundos")
    return result


In [0]:

def clean_dataframe(df):
    df["amount"] = df["amount"].str.replace("$", "", regex=True).astype(float)
    df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d %H:%M:%S")
    df["use_chip"] = df["use_chip"].replace({"Swipe Transaction": "False", "Online Transaction": "True"})
    return df

In [0]:
def sort_dataframe(df):
    return df.sort_values(by="amount", ascending=False)

In [0]:
# Função para agrupar o DataFrame
# Exemplo: Contar quantos cartões de cada marca existem
def group_dataframe(df):
    return df.groupby("card_brand").size().reset_index(name="count")

In [0]:
def group_dataframe(df):
    return df.groupby("merchant_state").size().reset_index(name="count")

In [0]:
# Função para filtrar o DataFrame
def filter_dataframe(df):
    return df[(df["use_chip"] == "True") & (df["amount"] > 100)]

In [0]:
def salvar_to_csv(df):
    df.to_csv('output.csv')

In [0]:
# Função para salvar o DataFrame
def save_file(df, path):
    df.to_csv(path, index=False)


In [0]:
output_path = 'output_path3.csv'

# Aplicar as transformações com medição de tempo
df = measure_time(read_file, file_content)
df_cleaned = measure_time(clean_dataframe, df)
df_sorted = measure_time(sort_dataframe, df_cleaned)
df_grouped = measure_time(group_dataframe, df_cleaned)
df_filtered = measure_time(filter_dataframe, df_cleaned)
measure_time(save_file, df_cleaned, output_path)

# Mostrar os resultados
print("Dados originais:")
print(df.head())

print("Dados limpos:")
print(df_cleaned.head())

print("Dados ordenados:")
print(df_sorted.head())

print("Dados agrupados:")
print(df_grouped.head())

print("Dados filtrados:")
print(df_filtered.head())

Tempo de execução para read_file: 42.2062 segundos
Tempo de execução para clean_dataframe: 18.4669 segundos
Tempo de execução para sort_dataframe: 7.1148 segundos
Tempo de execução para group_dataframe: 1.1413 segundos
Tempo de execução para filter_dataframe: 2.1660 segundos
Tempo de execução para save_file: 64.7187 segundos
Dados originais:
        id                date  client_id  card_id  amount use_chip  \
0  7475327 2010-01-01 00:01:00       1556     2972  -77.00    False   
1  7475328 2010-01-01 00:02:00        561     4575   14.57    False   
2  7475329 2010-01-01 00:02:00       1129      102   80.00    False   
3  7475331 2010-01-01 00:05:00        430     2860  200.00    False   
4  7475332 2010-01-01 00:06:00        848     3915   46.41    False   

   merchant_id merchant_city merchant_state      zip   mcc errors  
0        59935        Beulah             ND  58523.0  5499    NaN  
1        67570    Bettendorf             IA  52722.0  5311    NaN  
2        27092         Vi